In [95]:
import pandas as pd
import numpy as np

data = r"3993249.csv"

df = pd.read_csv(data)
df.set_index('DATE', inplace=True)

stations = df['STATION'].unique()
for station in stations:
    if df[df['STATION'] == station]['PRCP'].sum() == 0:
        df = df[df['STATION'] != station]
stations = df['STATION'].unique()

print(stations)

# df = df[df['STATION'] != 'USR0000NBES'] # get rid of Bessey Station, it doesn't have much data
# stations = df['STATION'].unique()

df = df[['STATION', 'PRCP', 'TMAX', 'TMIN', 'AWND', 'WDF1', 'WDF2', 'WSF1', 'WSF2']]

df['PRCP'] *= 100.0

df = df.pivot(columns='STATION', values=['PRCP', 'TMAX', 'TMIN', 'AWND', 'WDF1', 'WDF2', 'WSF1', 'WSF2'])
df = df.loc['1984-01-01':'2024-12-31']
df.columns = ['_'.join(col) for col in df.columns]

#print(df)

df = df.ffill().fillna(0)

#df[['WDF1', 'WDF2', 'WSF1', 'WSF2', 'AWND']] = df[['WDF1', 'WDF2', 'WSF1', 'WSF2']].ffill().fillna(0)
for station in stations:
    df[f'WSF12_{station}'] = df[f'WSF1_{station}'] + df[f'WSF2_{station}']
    df[f'WDF12_{station}'] = np.pi * (df[f'WDF1_{station}'] + df[f'WDF2_{station}']) / 2
    df[f'UWSF12_{station}'] = df[f'WSF12_{station}'] * np.cos(df[f'WDF12_{station}'])
    df[f'VWSF12_{station}'] = df[f'WSF12_{station}'] * np.sin(df[f'WDF12_{station}'])
    df[f'UWDF12_{station}'] = np.cos(df[f'WDF12_{station}']).pow(2)
    df[f'VWDF12_{station}'] = np.sin(df[f'WDF12_{station}']).pow(2)
    df = df.drop([f'WDF1_{station}', f'WDF2_{station}', f'WSF1_{station}', f'WSF2_{station}'], axis=1)

    df[f'TAVG_{station}'] = (df[f'TMAX_{station}'] + df[f'TMIN_{station}'])/2
    df[f'TSPREAD_{station}'] = (df[f'TMAX_{station}'] - df[f'TMIN_{station}'])
    df[f'WSPREAD_{station}'] = df[f'WSF12_{station}'] - df[f'AWND_{station}']

new_column_order = []
for station in stations:
    new_column_order += [f'PRCP_{station}', f'TMAX_{station}', f'TMIN_{station}', f'AWND_{station}', f'UWSF12_{station}', f'VWSF12_{station}']
for station in stations:
    new_column_order += [f'WSF12_{station}', f'WDF12_{station}', f'UWDF12_{station}', f'VWDF12_{station}', f'TAVG_{station}', f'TSPREAD_{station}', f'WSPREAD_{station}']
df = df[new_column_order]

df.insert(0, 'DAY', pd.to_datetime(df.index).day_of_year)
df['DAY'] = df['DAY'].apply(lambda x: x if x < 366 else 1)
df['DAY365'] = df['DAY'] / 365.25
df['SIN'] = np.sin(2 * np.pi * df['DAY365'])
df['COS'] = np.cos(2 * np.pi * df['DAY365'])

df.to_csv('multidata2.csv')

limited = df.loc[:'2000-01-01']

normals = limited.groupby('DAY').agg(['mean', 'std'])
normals.columns = ['_'.join(col) for col in normals.columns]
normals.to_csv('multinormals2.csv')

['USW00014935' 'USC00254335' 'USC00255810' 'USC00251825']
